# Initialization vs Isometry in Qiskit

According to `201 Tips and tricks.ipynb` there are exist two approaches to prepare the state: one was proposed by Shende, Bullock and Markov in ["Synthesis of Quantum Logic Circuits"](https://arxiv.org/abs/quant-ph/0406176) (2004) and another — by Iten et al. in ["Quantum Circuits for Isometries"](https://arxiv.org/abs/1501.06911) (2020). For more details please refer to correspondent notebook and original papers.

This notebook concerns comparison of these two methods in terms of how close obtained states are to the desired vector. For someone who knows quantum programming inside out, the answer might appear trivial, but not for me...

### Prerequisites

In [1]:
import math
import numpy as np

from qiskit import QuantumCircuit, BasicAer, execute, transpile
from scipy.stats import wasserstein_distance as ws
from numpy import dot
from numpy.linalg import norm

import warnings
warnings.filterwarnings("ignore")

### Metrics

Let's construct a class that will examine how emperical distribution is close to input vector in terms of [mean-absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error), [Wasserstein metric](https://en.wikipedia.org/wiki/Wasserstein_metric) and [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity).

In [2]:
class Metrics:
    
    def __init__(self):
        self.metrics_available = ["mae", "wasserstein", "cosine_similarity"]
    
    def compute(self, data, dist):
        metrics = dict(
            mae = np.abs(data - dist).mean(),
            wasserstein = ws(data, dist),
            cosine_similarity = np.dot(data, dist) / (np.linalg.norm(data) * np.linalg.norm(dist))
        )
        return metrics

### Bakends

We might want to use both simulation and real QPU for computation, so it makes sense to unify the interface of these.

In [3]:
class BackendFactory:
    
    def __init__(self):
        pass
    
    def get_new(self):
        pass
    
    def __str__(self):
        pass

In [4]:
class QASMSimulatorFactory():
    
    def __init__(self):
        super().__init__()
        pass
    
    def get_new(self):
        return BasicAer.get_backend('qasm_simulator')
    
    def __str__(self):
        return 'qasm_simulator'

In [5]:
class Experiment:
    
    def __init__(self, backend_factory, n_qubits=4, shots=65535):
        self.backend_factory = backend_factory
        self.n = n_qubits
        self.vector_size = 2 ** n_qubits
        self.shots = shots
        self.metrics = Metrics()
        
    def run(self):
        data = self.__get_random_vector()
        
        init_qc = self.__prepare_initialize_circuit(data)
        isom_qc = self.__prepare_isometry_circuit(data)

        result = dict(
            initialize = self.__apply(init_qc, data, self.shots),
            isometry = self.__apply(isom_qc, data, self.shots)
        )
        return result
    
    def __prepare_initialize_circuit(self, data):
        n = self.n
        range_n = range(n)
        qc = QuantumCircuit(n, n)
        qc.initialize(data)
        qc.measure(range_n, range_n)
        return qc
    
    def __prepare_isometry_circuit(self, data):
        n = self.n
        range_n = range(n)
        qc = QuantumCircuit(n, n)
        qc.isometry(data, list(range_n), None)
        qc.measure(range_n, range_n)
        return qc
    
    def __apply(self, qc, data, shots):
        counts = execute(qc, self.backend_factory.get_new(), shots=shots).result().get_counts()
        dist = self.__get_dist(counts)
        metrics = self.metrics.compute(data, dist)
        return metrics
    
    def __get_dist(self, counts):
        dist = [0. for i in range(self.vector_size)]
        for a, b in sorted(list(counts.items())):
            dist[int(a, 2)] = b
        dist = np.array(dist)
        dist = dist / np.linalg.norm(dist)
        return dist
        
    def __get_random_vector(self):
        a = np.random.rand(self.vector_size)
        a = a / np.linalg.norm(a)
        return a

In [6]:
experiment = Experiment(
    backend_factory=QASMSimulatorFactory(),
    n_qubits=4, 
    shots=10000
)

In [7]:
experiment.run()

{'initialize': {'mae': 0.04444438063046702,
  'wasserstein': 0.044444380630467024,
  'cosine_similarity': 0.981895760070583},
 'isometry': {'mae': 0.04389583423177455,
  'wasserstein': 0.04389583423177455,
  'cosine_similarity': 0.980759435140746}}

### Analyse multiple experiments

In [8]:
class Inspector:
    
    def __init__(self, backend_factory, n_qubits=4, shots=65535, iterations=100):
        self.backend_factory = backend_factory
        self.n_qubits = n_qubits
        self.shots = shots
        self.iterations = iterations
        self.metrics_available = Metrics().metrics_available
    
    def run(self):
        metrics_dict_init = {m: list() for m in self.metrics_available}
        metrics_dict_isom = {m: list() for m in self.metrics_available}
        for i in range(self.iterations):
            e = Experiment(
                backend_factory=self.backend_factory,
                n_qubits=self.n_qubits,
                shots=self.shots
            )
            e = e.run()
            
            for m, v in e["initialize"].items():
                metrics_dict_init[m].append(v)
                
            for m, v in e["isometry"].items():
                metrics_dict_isom[m].append(v)
        
        print(f"Backend: {self.backend_factory}")
        
        print("  'qc.initialise':")
        for m, v in metrics_dict_init.items():
            v = np.array(v)
            print(f"\t{m}: {v.mean():.8f} ± {v.var():.8f}")
        
        print("  'qc.isometry':")
        for m, v in metrics_dict_isom.items():
            v = np.array(v)
            print(f"\t{m}: {v.mean():.8f} ± {v.var():.8f}")
         
        

In [9]:
inspector = Inspector(
    backend_factory=QASMSimulatorFactory(),
    n_qubits=5, 
    shots=65535, 
    iterations=10
)
inspector.run()

Backend: qasm_simulator
  'qc.initialise':
	mae: 0.04015251 ± 0.00002486
	wasserstein: 0.04014760 ± 0.00002484
	cosine_similarity: 0.96653506 ± 0.00006403
  'qc.isometry':
	mae: 0.04011296 ± 0.00002694
	wasserstein: 0.04011296 ± 0.00002694
	cosine_similarity: 0.96639625 ± 0.00006414
